In [52]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Union

class ImagePreprocessing :
    """preprocess the image and the bounding boxes"""
    def __init__(self):
        """Constructor for the class
        parameters:
        -----------
        bboxesImages: list
             a list of cropped bounding box images.
        topLeft :torch.Tensor
             a tensor of top-left corner coordinates of the bounding boxes.
        originalDim : torch.Tensor
             a tensor of original dimensions (width, height) of the bounding boxes.
        normalizeBboxes :torch.Tensor
             a tensor of normalized bounding box images.
        """
        self.bboxesImages=None
        self.topLeft=None
        self.originalDim=None
        self.normalizeBboxes=None

    def loadImage(self, imgPath: str):
        """Load an image from a file as a numpy array.
        Parameters:
        -----------
            img_path: str
                Path to the image file.
        Returns:
        --------
            image: np.ndarray
                Loaded image as a numpy array.
        """
        image = plt.imread(imgPath)
        return torch.from_numpy(np.array(image))

    def cropImage(self,img:torch.Tensor,bboxes:List[List[int]]):
        """Crop the image according to the bounding boxes.
        Parameters:
        -----------
            img: np.ndarray
                Image to be cropped.
            bboxes: np.ndarray
                Bounding boxes to crop the image.
        """
        bbox=[]
        bboxesImages=[]
        topLeft=[]
        originalDim=[]
        for box in bboxes:
            x1, y1= box[0], box[1]
            x2,y2= box[2], box[3]
            topLeft.append((x1,y1))
            bbox=img[y1:y2, x1:x2]
            originalDim.append((bbox.shape[1],bbox.shape[0]))
            bboxesImages.append(bbox)

        self.topLeft=torch.from_numpy(np.array(topLeft))
        self.originalDim=torch.from_numpy(np.array(originalDim))
        self.bboxesImages=bboxesImages
        
    def _resizeNormalizeBboxes(self,width:int,height:int):
        """Resize and normalize the bounding boxes.
        Parameters:
        -----------
            width: int
                Width of the resized image.
            height: int
                Height of the resized image.
        """
        transform = transforms.Compose([
        transforms.Resize((width, height)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) 
        self.resizedBboxes = self.bboxesImages[0].float().permute(2,0,1)
        self.resizedBboxes=transform(self.resizedBboxes/255).unsqueeze(0)

        for img in self.bboxesImages[1:]:
            img = img.permute(2,0,1).float()
            img = transform(img/255)
            self.resizedBboxes = torch.cat((self.resizedBboxes, img.unsqueeze(0)), dim=0)

            
    def preprocessBboxes(self, img, width:int ,height:int, bboxes:List[List[int]]):
        """Preprocess the bounding boxes.
        Parameters:
        -----------
            imgPath: str
                Path to the image file.
            width: int
                Width of the resized image.
            height: int
                Height of the resized image.
            bboxes: np.ndarray
                Bounding boxes to crop the image.
        Returns:
        --------
            resized_bboxes: torch.Tensor
                Tensor of normalized bounding box images.
        """
        #img=self.loadImage(imgPath)
        self.cropImage(img,bboxes)
        self._resizeNormalizeBboxes(width,height)
        return self.resizedBboxes
    
    def getKeypoints(self,keypoints_ratio):
        ''''''
        

In [59]:
image_proccess=Image_Preprocessing()
bboxes =  torch.tensor([[1, 61, 171, 569],
              [75, 57, 241, 472],
              [155, 65, 266, 407],
              [224, 64, 335, 360]])
# keypoints_ratio =  torch.tensor([[1,2,1,2,4,6],[2,5,9,9,3],[[1,2],[1,2],[1,2]])

ready_images=image_proccess.preprocessBboxes("fsoco_sample/bounding_boxes/121.jpeg",50,80,bboxes)
# print(image_proccess.originalDim)
# print(len(ready_images))
# print(image_proccess.topLeft)
# print(ready_images.shape)
# # print(ready_images)